In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# A function to retrieve the data types for each column in a Pandas DataFrame

def get_metadata(df):
    metadata = {}
    unique_count = []
    for (col, dtype, null_count) in zip(df.columns, df.dtypes, df.isnull().sum()):
        metadata[col] = (dtype, null_count)
        unique_count.append(df[col].nunique())
    
    columns = []
    dtypes = []
    nulls = []
    for key,value in metadata.items():
        columns.append(key)
        dtypes.append(value[0])
        nulls.append(value[1])
    
    data = {"column_name":columns,"data_type":dtypes, "null_count":nulls, "unique_count":unique_count}
    df_metadata = pd.DataFrame(data)
    return df_metadata

In [ ]:
df = pd.read_csv("data/day.csv")

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['dteday'] = pd.to_datetime(df['dteday'], format='%d-%m-%Y')

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(df['dteday'], df['temp'], label = 'temp')
plt.plot(df['dteday'], df['atemp'], label = 'atemp')
#plt.plot(df['dteday'], df['cnt'], label = 'cnt')
plt.legend()
plt.show()

In [ ]:
df_meta = get_metadata(df)
df_meta

In [ ]:
categorical = df_meta[df_meta['unique_count'] <= 12]
categorical = categorical['column_name'].to_list()

In [ ]:
for col in categorical:
    df[col] = df[col].astype(str)

In [ ]:
df_meta = get_metadata(df)
df_meta

In [ ]:
import seaborn as sns

plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), cmap='YlGnBu', annot = True)
plt.show()

In [ ]:
df['cnt_to_ registered_ratio'] = df['cnt']/df['registered']
df['cnt_to_ casual_ratio'] = df['cnt']/df['casual']

In [ ]:
df_pivot = df.pivot_table(values = 'cnt', index = 'season', columns = 'yr', aggfunc = 'mean')

In [ ]:
df_pivot['perc_change'] = df_pivot['1'].divide(df_pivot['0']).multiply(100)

In [ ]:
df_pivot

In [ ]:
df_pivot = df.pivot_table(values = 'cnt', index = 'season', columns = 'yr', aggfunc = 'mean')

In [ ]:
df_pivot

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error